In [ ]:
'''
    Title         : Searching Test Suite for pyAwake
    Author        : Aman Singh Thakur
    Last Edited   : 08 August, 2019
'''


import unittest
import pyawake
from pyawake.Searching import Searching
from pyawake.Visualizing import Visualizing
import h5py
import datetime
from datetime import datetime

search_ptr = Searching()
visualize_ptr = Visualizing()

central_csv_name = "CentralCSV.csv"
dataset_txt_name = "dataset_hash.csv"
comment_csv_name = "comment_hash.csv"
fieldnames = ['DatasetName', 'GroupName', 'AcqStamp',  'Exception', 'Shape', 'Datatype']
dir_h5_filepath = "/eos/experiment/awake/event_data/"
dir_home = "/eos/experiment/awake/event_data/"
dir_csv_filepath = "/eos/experiment/awake/CSVFiles/"
dir_csv_central_filepath = "/eos/experiment/awake/CSVFiles/Central/"
central_csv_name = "CentralCSV.csv"
image_filepath = "/eos/user/a/amthakur/PNGFiles/"
streak_dataset_cuts = "/AwakeEventData/TT41.BCTF.412340/Acquisition/totalIntensityPreferred < 100.0,/AwakeEventData/TT41.BCTF.412340/Acquisition/totalIntensityPreferred > 1.0"

dataset_cuts = "/AwakeEventData/TT41.BCTF.412340/Acquisition/totalIntensityPreferred < 100.0,/AwakeEventData/TT41.BCTF.412340/Acquisition/totalIntensityPreferred > 1.0"

In [ ]:
class TestNotebook(unittest.TestCase):

    def test_open_file(self):
        df=search_ptr.open_file('/eos/experiment/awake/CSVFiles/2018/05/13/1526218206135000000_117_613.csv')
        self.assertEqual(False, df.empty)
    
    def test_get_all_files(self):
        all_files = search_ptr.get_all_files('/eos/experiment/awake/CSVFiles/2018/05/13/','csv')
        self.assertEqual(2055, len(all_files))
    
    def test_convert_unix_time_to_datetype(self):
        utc_dt, cern_dt, file_ptr1, file_ptr2 = search_ptr.convert_unix_time_to_datetype('1526218206135000000_117_613.csv')
        self.assertEqual(1526218206.0, cern_dt.timestamp())
        
    def test_decode_attr(self):
        attr = search_ptr.decode_attr('/0x8')
        self.assertEqual('/0x8', attr)
    
    def test_get_dataset_attr(self):
        attr = {}
        attr['key1'] = "value1"
        attr['key2'] = "value2"
        str_attr = search_ptr.get_dataset_attr(attr)
        self.assertEqual("{'key1': 'value1', 'key2': 'value2'}", str_attr)
    
    def test_get_dataset_value(self):
        h5File = h5py.File('/eos/experiment/awake/event_data/2018/05/13/1526218206135000000_117_613.h5', 'r')
        h5File = h5File['/AwakeEventData/XMPP-STREAK/StreakImage/streakImageWidth']
        response = search_ptr.get_dataset_value(1, h5File)
        self.assertEqual('672', response)
        
    def test_convert_csv_to_h5(self):
        response = search_ptr.convert_csv_to_h5('1526218206135000000_117_613.csv')
        self.assertEqual('1526218206135000000_117_613.h5', response)
        
    def test_convert_str_to_timestamp(self):
        self.assertEqual(1562926320.0, search_ptr.convert_str_to_timestamp('2019-07-12 12:12:00').timestamp())
        
    def test_column_data(self):
        Timestamp = search_ptr.convert_str_to_timestamp('2019-07-12 12:12:00').timestamp()
        h5_filename = '1526218206135000000_117_613.h5'
        csv_filename = '/eos/experiment/awake/CSVFiles/2018/05/13/1526218206135000000_117_613.csv'
        dataset_name = ['/AwakeEventData/XMPP-STREAK/StreakImage/streakImageWidth', '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageHeight']
        column_data_retrieved = search_ptr.get_column_data(h5_filename, csv_filename, Timestamp, dataset_name, dataset_cuts)
        self.assertEqual(4, len(column_data_retrieved))
        
    def test_get_metadata(self):
        Timestamp = search_ptr.convert_str_to_timestamp('2019-07-12 12:12:00').timestamp()
        h5_filename = '1526218206135000000_117_613.h5'
        csv_filename = '/eos/experiment/awake/CSVFiles/2018/05/13/1526218206135000000_117_613.csv'
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        metadata = search_ptr.get_metadata(h5_filename, csv_filename, Timestamp, dataset_name, dataset_cuts)
        self.assertEqual(17, len(metadata.keys()))
        
    def test_get_dataset(self):
        Timestamp = search_ptr.convert_str_to_timestamp('2019-07-12 12:12:00').timestamp()
        h5_filename = '/eos/experiment/awake/event_data/2018/05/13/1526218206135000000_117_613.h5'
        csv_filename = '/eos/experiment/awake/CSVFiles/2018/05/13/1526218206135000000_117_613.csv'
        dataset_name = '/AwakeEventData/BOVWA.04TT41.CAM4/ExtractionImage/imageRawData'
        dataset = search_ptr.get_dataset(h5_filename, csv_filename, Timestamp, dataset_name, dataset_cuts)
        self.assertEqual(25, len(dataset.keys()))
        
    def test_get_all_dates(self):
        start = datetime.strptime("2018-05-01 00:00:00", '%Y-%m-%d %H:%M:%S')
        end = datetime.strptime("2018-05-31 23:59:59", '%Y-%m-%d %H:%M:%S')
        all_dates = search_ptr.get_all_dates(start, end)
        self.assertEqual(31, len(all_dates))
        
    def test_get_all_files_tree(self):
        start_date = datetime.strptime("2018-05-01 00:00:00", '%Y-%m-%d %H:%M:%S')
        end_date = datetime.strptime("2018-05-31 23:59:59", '%Y-%m-%d %H:%M:%S')
        dir_csv_filepath = '/eos/experiment/awake/CSVFiles/'
        all_files = search_ptr.get_all_files_tree(dir_csv_filepath, start_date, end_date, "csv")
        self.assertEqual(16536, len(all_files))
    
    def test_generate_stmts_cuts(self):
        all_stmt_cuts = search_ptr.generate_stmts_cuts(dataset_cuts)
        self.assertEqual(2, len(all_stmt_cuts))
        
    def test_apply_cuts(self):
        cuts_arr = search_ptr.generate_stmts_cuts(dataset_cuts)
        df=search_ptr.open_file('/eos/experiment/awake/CSVFiles/2018/05/13/1526218206135000000_117_613.csv')
        self.assertEqual(True, search_ptr.apply_cuts('DatasetName', cuts_arr, df))
    
    def test_get_all_dataset_files_by_timestamp(self):
        dir_csv_filepath = '/eos/experiment/awake/CSVFiles/'
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        starting_timestamp = '2018-05-28 12:00:00'
        ending_timestamp = '2018-05-28 23:59:59'
        all_datasets = search_ptr.get_all_dataset_files_by_timestamp(dir_csv_filepath, dataset_name, 'DatasetName', starting_timestamp, ending_timestamp)
        self.assertEqual(0, len(all_datasets))
    
    def test_get_dict_from_str(self):
        dict = search_ptr.get_dict_from_str("{'key1': 'value1', 'key2': 'value2'}")
        self.assertEqual('value1',dict['key1'])
        
    def test_load_column_timestamp(self):
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        starting_timestamp = '2018-05-28 12:00:00'
        ending_timestamp = '2018-05-28 15:59:59'
        values = search_ptr.load_column_timestamp(dir_h5_filepath, dir_csv_filepath, dataset_name, starting_timestamp, ending_timestamp, dataset_cuts)
        self.assertEqual(0, len(values))
    
    def test_load_group_metadata_timestamp(self):
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        starting_timestamp = '2018-05-28 12:00:00'
        ending_timestamp = '2018-05-28 15:59:59'
        values = search_ptr.load_group_metadata_timestamp(dir_h5_filepath, dir_csv_filepath, dataset_name, starting_timestamp, ending_timestamp, dataset_cuts)
        self.assertEqual(0, len(values))
    
    def test_search_by_dataset_timestamp(self):
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        starting_timestamp = '2018-05-28 12:00:00'
        ending_timestamp = '2018-05-28 15:59:59'
        values = search_ptr.search_by_dataset_timestamp(dir_h5_filepath, dir_csv_filepath, dataset_name, starting_timestamp, ending_timestamp, dataset_cuts)
        self.assertEqual(0, len(values))
        
    def test_load_group_metadata_timestamp(self):
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        starting_timestamp = '2018-05-28 12:00:00'
        ending_timestamp = '2018-05-28 15:59:59'
        values = search_ptr.load_group_metadata_timestamp(dir_h5_filepath, dir_csv_filepath, dataset_name, starting_timestamp, ending_timestamp, dataset_cuts)
        self.assertEqual(0, len(values))
    
    def test_search_by_dataset_timestamp(self):
        dataset_name = '/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
        starting_timestamp = '2018-05-28 12:00:00'
        ending_timestamp = '2018-05-28 15:59:59'
        values = search_ptr.search_by_dataset_timestamp(dir_h5_filepath, dir_csv_filepath, dataset_name, starting_timestamp, ending_timestamp, dataset_cuts)
        self.assertEqual(0, len(values))
        
    def test_load_comment_dataset(self):
        dataset_txt, comment_csv = search_ptr.load_comment_dataset(dir_csv_central_filepath)
        self.assertEqual(2513, len(dataset_txt)+len(comment_csv))

    def test_search_dataset_name(self):
        dataset_txt, comment_csv = search_ptr.load_comment_dataset(dir_csv_central_filepath)
        values = search_ptr.search_dataset_name(dataset_txt, 'streak')
        self.assertEqual(28, len(values))
    
    def test_search_dataset_comment(self):
        dataset_txt, comment_csv = search_ptr.load_comment_dataset(dir_csv_central_filepath)
        self.assertEqual({'/AwakeEventData/AIRTR01/Acq/numericPosition'},search_ptr.search_dataset_comment(comment_csv, 'mm'))

    def test_load_column(self):
        dataset = search_ptr.load_column("streak xmpp", "", "2018-05-12 12:00:00", "2018-05-12 23:59:59", streak_dataset_cuts)
        self.assertEqual(620, len(dataset)) #Choose any column and put 0-1124
        
    def test_search(self):
        dataset = search_ptr.search("streak xmpp ", "", "2018-05-12 18:00:00", "2018-05-12 23:59:59", streak_dataset_cuts)
        self.assertEqual(0, len(dataset)) #0-3

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)